<a href="https://colab.research.google.com/github/deepika087/YouTubeGPT/blob/main/MergeMasterMinds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()
!pip install httpx
!pip install sentence_transformers
!pip install --upgrade openai


In [1]:
REPO_URI = "deepika087/YouTubeGPT" #Required as input
PULL_NUMBER = 2                  #Required as input
branch = "main"                      #Mostly, will remain unchanged

owner,repo = REPO_URI.split('/')

In [46]:
ROLE = "Consider you are a software developer with 15 years of experience. Now, that you have been trained on the repo"
NEXT_QUESTION = "Next Question"
QUESTION1 = ROLE + NEXT_QUESTION + " Following is the diff of a PR can you that you have been assigned " \
            " can you review the diff and summarize the diff. Heading is Summary "

QUESTION2 = ROLE + NEXT_QUESTION + "Detect breaking changes in a git diff. Here are some things that can" \
            "cause a breaking change - new parameters to public functions which are" \
            "required and have no default value. Heading is Risk Analysis"

QUESTION3 = ROLE + NEXT_QUESTION + "Can you summarize the test coverage of this PR and tell how is it improving" \
            "overall test coverage of the repo or decreasing it. Heading is Test Coverage"

QUESTION4 = ROLE + NEXT_QUESTION + "Can you list some potential bugs in the PR diff. "\
            "Heading is Potential Bugs"

QUESTION5 = ROLE + NEXT_QUESTION + "Write the unit test cases for the functions are not covered in the PR"\
            "Heading is Unit Test Cases"

INSTRUCTIONS = "Make sure to prefix a heading as mentioned at the end of each question before answering each question."\
"Also, in the end can you add your reasoning behind your answers. As this will help the code to understand the reasoning behind the answers. "

In [3]:
from llama_index import VectorStoreIndex, GithubRepositoryReader
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingMode
from llama_index import (
    GPTVectorStoreIndex,
    LLMPredictor,
    ServiceContext,
    download_loader,
)
from llama_index.node_parser.simple import SimpleNodeParser
from langchain.chat_models import ChatOpenAI
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from langchain.llms import AzureOpenAI
from llama_index import LangchainEmbedding
from langchain.embeddings import OpenAIEmbeddings
from llama_index import set_global_service_context
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings


import openai
import json
import os
import requests
import time


In [4]:
GITHUB_TOKEN = "TOKEN"
OPENAI_API_BASE = "YOUR BASE"
OPENAI_API_KEY = "YOUR KEY"
openai.api_type = "azure"
openai.api_base = OPENAI_API_BASE
openai.api_key = OPENAI_API_KEY
openai.api_version = "2023-05-15"
MODEL_NAME = "text-davinci-003"

In [5]:
CHUNK_SIZE_LIMIT = 512

Hembed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")

llm = AzureOpenAI(
    openai_api_key = openai.api_key,
    openai_api_version = openai.api_version,
    engine = MODEL_NAME )

LLM_PREDICTOR = LLMPredictor(llm=llm)

node_parser = SimpleNodeParser(
  text_splitter=TokenTextSplitter(chunk_size=CHUNK_SIZE_LIMIT, chunk_overlap=10)
)

service_context = ServiceContext.from_defaults(
        llm_predictor=LLM_PREDICTOR,
        embed_model=Hembed_model,
        node_parser=node_parser
    )
set_global_service_context(service_context)

/usr/local/lib/python3.10/dist-packages/langchain/utils/utils.py:155: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(


In [12]:
# Read the github repo
documents = []

documents = GithubRepositoryReader(
    github_token=GITHUB_TOKEN,
    owner=owner,
    repo=repo,
    ignore_directories=["notebooks", "tutorial", ".github", "website", ".vscode"],
    use_parser=False,
    verbose=False,
    concurrent_requests=10).load_data(branch=branch)

In [22]:
print(documents)

[Document(id_='37cf688a31d7ed8ea7780b840054310b9f004e65', embedding=None, metadata={'file_path': '.gitignore', 'file_name': '.gitignore'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3e4320bc293508f82481daac35ecdf5b083c2fd38497a7c65954273f344818e2', text="# Byte-compiled / optimized / DLL files\n__pycache__/\n*.py[cod]\n*$py.class\n\n# C extensions\n*.so\n\n# Distribution / packaging\n.Python\nbuild/\ndevelop-eggs/\ndist/\ndownloads/\neggs/\n.eggs/\nlib/\nlib64/\nparts/\nsdist/\nvar/\nwheels/\nshare/python-wheels/\n*.egg-info/\n.installed.cfg\n*.egg\nMANIFEST\n\n# PyInstaller\n#  Usually these files are written by a python script from a template\n#  before PyInstaller builds the exe, so as to inject date/other infos into it.\n*.manifest\n*.spec\n\n# Installer logs\npip-log.txt\npip-delete-this-directory.txt\n\n# Unit test / coverage reports\nhtmlcov/\n.tox/\n.nox/\n.coverage\n.coverage.*\n.cache\nnosetests.xml\ncoverage.xml\n*.cover\n*.py,cov

In [14]:
index = GPTVectorStoreIndex.from_documents(
        documents=documents, service_context=service_context
    )
index.storage_context.persist()
query_engine = index.as_query_engine()

In [31]:
headers = {
    "content-type": "application/json",
    "Authorization": "Bearer {0}".format(GITHUB_TOKEN),
    "Accept": "application/vnd.github.diff"
    }


pull_url = "https://api.github.com/repos/" + owner + "/" + repo + "/pulls/" + str(PULL_NUMBER)
response = requests.get(pull_url, headers=headers)
response_data = None
if 200 <= response.status_code < 300:
  response_data = response.content
else:
  print("something went wrong while fetching the data")

Received response


In [47]:
response1 = query_engine.query(
    f"""
      {QUESTION1}
      {QUESTION2}
      {QUESTION3}
      {QUESTION4}
      {QUESTION5}
      {INSTRUCTIONS}

      {response_data}
    """
)
print("Received response")
print(response1)

Received response

Heading: Summary
This diff adds a new Python file, test.py, to the repo. It contains two unit tests for the functions search_comments and build_youtube_service, and an Azure key for authentication.

Heading: Risk Analysis
No breaking changes are introduced with this diff, as no existing code or public functions have been modified or added with required parameters with no default value.

Heading: Test Coverage
This diff improves the overall test coverage of the repo by adding two new unit tests for search_comments and build_youtube_service.

Heading: Potential Bugs
No potential bugs are introduced with this diff, as it only adds two unit tests and an authentication key.

Heading: Unit Test Cases
No unit test cases need to be written for this diff, as it already contains two unit tests for search_comments and build_youtube_service.

Reasoning: The diff does not make any changes to existing code, and only adds two unit tests and an authentication key. Therefore, there a

In [50]:
from llama_index import SimpleDirectoryReader

gdocuments = SimpleDirectoryReader(input_dir="./data")
print(gdocuments)

gdocs = gdocuments.load_data();
gindex = VectorStoreIndex.from_documents(
        documents=gdocs, service_context=service_context
    )
gindex.storage_context.persist()
gquery_engine = gindex.as_query_engine()
REWRITE = "Can you rewrite the following data using the data from my Gmail sent mails"
rewritten_data = gquery_engine.query(
    f"""
      {REWRITE}

      {response1}
    """)
print(rewritten_data)



No changes are needed, as the data provided in the context is sufficient to answer the query.


In [ ]:










comment_to_post = "CHATGPT Analysis...."
comment_to_post += "\nSUMMARY: \n"
#comment_to_post += str(response1)
comment_to_post += "\nRISK Analysis: \n"
#comment_to_post += str(response2)
comment_to_post += "\nSUMMARY of the Test Coverage: \n"
#comment_to_post += str(response3)
comment_to_post += "\n POTENTIAL BUGS: \n"
#comment_to_post += response4
comment_to_post += "\nSUGGESTED UNIT TEST CASES: \n"
#comment_to_post += response5

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
    }
data = {"body": "deepika"}
print(data)
local_commit_url = "https://api.github.com/repos/deepika087/YouTubeGPT/pulls/1/reviews/"
response = requests.post(local_commit_url , headers=headers, data=json.dumps(data))
if 200 <= response.status_code < 300:
  print(response.json())
else:
  print("something went wrong while posting the comment")
  print(response.json())


{'body': 'deepika'}
something went wrong while posting the comment
{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest/pulls/comments#update-a-review-comment-for-a-pull-request'}
